# Bipolar disorder and response to lithium: blood

Dataset Files: 
* gds5393.csv 
* meta-gds5393.csv

**Introdução**

O transtorno bipolar é uma condição mental caracterizada por episódios de humor altamente variáveis, com períodos de euforia ou mania (conhecidos como episódios maníacos) alternando com períodos de profunda depressão (conhecidos como episódios depressivos). O tratamento para o transtorno bipolar geralmente inclui terapia e medicação.

Um dos medicamentos mais comuns utilizados para tratar o transtorno bipolar é o lítio. O lítio é um mineral que atua como estabilizador de humor e é eficaz na prevenção de episódios maníacos e depressivos em pessoas com transtorno bipolar. Ele funciona ajudando a equilibrar os níveis de determinados químicos no cérebro chamados neurotransmissores, que são responsáveis por transmitir informações entre as células cerebrais.


**Sobre o conjunto de dados**

O [conjunto de dados](https://www.ncbi.nlm.nih.gov/sites/GDSbrowser?acc=GDS5393) contém 120 amostras e é referente à análise de sangue periférico de pacientes com transtorno bipolar antes e 1 mês após o tratamento com lítio. A resposta dos pacientes ao lítio foi avaliada após 6 meses. Os resultados identificam uma assinatura de expressão gênica para a resposta ao tratamento com lítio em pacientes com transtorno bipolar.

Pela análise da [documentação](https://www.ncbi.nlm.nih.gov/geo/tools/profileGraph.cgi?ID=GDS5393) do conjunto de dados verificamos o metadados contém as seguintes colunas:
* **sample**: (GSM1105…) - Identificador da amostra. Referente a 120 amostras da colheita de sangue.
* **agent**: (control ou lithium) - Indica se a amostra é de um indivíduo de controlo ou se foi submetido ao tratamento com lítio.
* **other**: (responder ou non-responder) - Identifica se a amostra é de um indivíduo que respondeu ou não ao tratamento.
* **time**: (baseline ou 1 month) - Indica o momento em que a amostra foi colhida.
* **individual**: Identificador único do indivíduo, que pode ser OPT_(N) para indivíduos de control e Li+OPT_(N) para indivíduos que receberam lítio. 
                  Cada indivíduo um tem duas amostras referentes ao momento da recolha (baseline e 1 month).
* **description**: Descrição adicional sobre cada amostra.


**Carregamento dados e metadados do dataset**

In [1]:
# Importar bibliotecas necessárias para o processamento
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn import preprocessing
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.metrics import accuracy_score

# Ler dados usando a lib pandas
data = pd.read_csv("gds5393.csv", sep=',', index_col = 0)
meta = pd.read_csv("meta-gds5393.csv", sep=',', index_col = 0)

#### Preparação dos dados

In [2]:
# remover os genes que têm todos os valores como NaN (dados nulos), para não atrapalharem esta análise
data.dropna(inplace = True)

In [3]:
# fazer a transporta para termos os dados na forma de uma matriz genes x amostras
data = data.transpose()

In [4]:
# Dimensao dos dados
data.values.shape

(120, 47323)

## **Aprendizagem supervisionada**

Aprendizagem supervisionada é um tipo de aprendizagem de máquina em que o algoritmo aprende com base nas características que o define, ou seja, dados que possuem um objetivo claro definido. O objetivo é prever a classe de um dado de entrada com base nas características desse dado e nas relações existentes com as classes dos dados de treino.

Nesta análise, vamos explorar a capacidade de prever as classes mais relevantes deste dataset e analisar se podemos realizar previsões eficazes:
* "other" - prever a resposta de um indivíduo ao tratamento com lithium
* "agent" - determinar se a amostra é relativa a um indivíduo em tratamento com litío ou de um indivíduo de controle
* "time" - identificar se a amostra é de baseline ou após 1 mês de tratamento com litío.

### Modelo de classificação para prever a resposta ao tratamento com lítio - classe 'other'

#### **Pré-processamento**

**Filtragem dos dados**

In [5]:
# Remover genes que variam pouco preservando as features com maior variabilidade
threshold = 0.01

from sklearn.feature_selection import VarianceThreshold

# Remover todas as colunas (genes) que possuem uma variabilidade menor ou igual a 0.5
var_threshold = VarianceThreshold(threshold=threshold)
x_high_variance = var_threshold.fit_transform(data)

# Obter os índices das colunas com alta variância
high_variance_columns = data.columns[var_threshold.get_support()]

# Criar um novo dataframe apenas com as colunas de alta variância
high_variance_data = data[high_variance_columns]
print(high_variance_data.shape)

(120, 22164)


Geralmente os modelos de aprendizagem máquina devem ser treinados com dados representativos da situação real, sem ruído ou informações irrelevantes. Nesse sentido, a filtragem dos dados é uma forma de alcançar esse objetivo, removendo informação que possa prejudicar a performance do modelo. 

As estratégias de filtragem dos dados, como VarianceThreshold, SelectKBest e Teste ANOVA, analisadas no ficheiro [tp_gds5393_data_analysis_multi](https://github.com/cvmota/BioInformatica/blob/main/tp_gds5393_data_analysis_multi.ipynb), são úteis para selecionar as melhores características ou atributos dos dados e usar como input para os modelos de treino, com o objetivo de remover características irrelevantes ou ruidosas.

No entanto, a filtragem dos dados também pode remover informações úteis dos dados e, neste caso, não temos informação sobre os intervalos de referência de cada um dos indicadores para avaliar a estratégia de filtragem mais adequada e garantir que o modelo tenha acesso às informações relevantes necessárias para realizar as previsões corretamente.

Nesse sentido, fizemos alguns testes com diferentes threshold e, tendo em conta também a analise no ficheiro [tp_gds5393_data_analysis_multi](https://github.com/cvmota/BioInformatica/blob/main/tp_gds5393_data_analysis_multi.ipynb), consideramos que na generalidade conseguimos melhores resultados com threshold = 0.01.

In [6]:
# standardização dos dados
input_sc = preprocessing.scale(high_variance_data)

**Divisão do conjunto de dados em dados para treino e dados para teste dos modelos.**

In [7]:
classe = 'other'
def splitData(classe):
    # Obter a lista única de indivíduos
    unique_individuals = np.unique(meta['individual'])

    # calcular o número de posições para treino e teste para obter mais ou menos 70% dos dados para treino e 30% para testar o modelo
    #
    num_individuals = len(unique_individuals)
    num_train = int(0.7 * num_individuals)
    num_test = num_individuals - num_train
    # print(num_test)

    # Criar um novo DataFrame de saída, com a standardizacao dos dados
    input_sc_df = pd.DataFrame(input_sc, index=meta['sample'], columns=high_variance_data.columns)

    # Dividir a lista em duas partes para obter os conjuntos de treino e de teste
    indices = np.random.permutation(len(unique_individuals))
    test_individuals = unique_individuals[indices[:-num_test]]
    train_individuals = unique_individuals[indices[-num_test:]]

    # Usar a lista de indivíduos de treino para filtrar os dados de treino e de teste
    train_indices = meta[meta['individual'].isin(train_individuals)].index
    train_in = input_sc_df.loc[train_indices]
    train_out = (meta.loc[train_indices])[classe].values

    test_indices = meta[meta['individual'].isin(test_individuals)].index
    test_in = input_sc_df.loc[test_indices]
    test_out = (meta.loc[test_indices])[classe].values
    
    print("Separação dos dados para treino e teste")
    print("Input shape:", input_sc.shape)
    print("Train in shape:", train_in.shape)
    print("Train out shape:", train_out.shape)
    print("Test in shape:", test_in.shape)
    print("Test out shape:", test_out.shape)
    return train_in, train_out, test_in, test_out

train_in, train_out, test_in, test_out = splitData(classe)

Separação dos dados para treino e teste
Input shape: (120, 22164)
Train in shape: (36, 22164)
Train out shape: (36,)
Test in shape: (84, 22164)
Test out shape: (84,)


#### **Modelação**
Criação de modelos supervisionados de aprendizagem máquina.

**Árvore de decisão**

In [8]:
from sklearn import tree

def tree_model_training(train_in, train_out, test_in, test_out):
    tree_model = tree.DecisionTreeClassifier()
    tree_model = tree_model.fit(train_in, train_out)
    tree_pred = tree_model.predict(test_in)
    print("\nÁrvore de decisão")
    print("Valores previstos: ", tree_pred)
    print("Valores reais: " , test_out)
    return tree_model, tree_pred

tree_model, tree_pred = tree_model_training(train_in, train_out, test_in, test_out)


Árvore de decisão
Valores previstos:  ['non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'responder' 'non-responder' 'responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'responder' 'non-responder' 'responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'respo

**SVM**

In [9]:
from sklearn import svm

def svm_model_training(train_in, train_out, test_in, test_out):
    clf = svm.SVC(gamma=0.001, C=100.)
    svm_model = clf.fit(train_in, train_out)
    svm_pred = clf.predict(test_in)
    print("\nSVM")
    print("Valores previstos: " , svm_pred)
    print("Valores reais: " , test_out)
    return svm_model, svm_pred

svm_model, svm_pred = svm_model_training(train_in, train_out, test_in, test_out)


SVM
Valores previstos:  ['non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'


**Naive Bayes**

In [10]:
from sklearn.naive_bayes import GaussianNB

def gnb_model_training(train_in, train_out, test_in, test_out):
    gnb_model = GaussianNB()
    gnb_model = gnb_model.fit(train_in, train_out)
    gnb_pred = gnb_model.predict(test_in)
    print("\nNaive Bayes")
    print("Valores previstos: " , gnb_pred)
    print("Valores reais: " , test_out)
    return gnb_model, gnb_pred

gnb_model, gnb_pred = gnb_model_training(train_in, train_out, test_in, test_out)


Naive Bayes
Valores previstos:  ['non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-res

**Regressão Logística**

In [11]:
from sklearn import linear_model

def logistic_model_training(train_in, train_out, test_in, test_out):
    logistic_model = linear_model.LogisticRegression(C=1e5, solver = "liblinear", multi_class = "auto")
    logistic_model = logistic_model.fit(train_in, train_out)
    logistic_pred =  logistic_model.predict(test_in)
    print("\nRegressão Logística")
    print("Valores previstos: " , logistic_pred)
    print("Valores reais: " , test_out)
    return logistic_model, logistic_pred

logistic_model, logistic_pred = logistic_model_training(train_in, train_out, test_in, test_out)


Regressão Logística
Valores previstos:  ['non-responder' 'non-responder' 'responder' 'non-responder' 'responder'
 'responder' 'responder' 'non-responder' 'responder' 'responder'
 'responder' 'responder' 'responder' 'non-responder' 'responder'
 'responder' 'responder' 'non-responder' 'non-responder' 'non-responder'
 'responder' 'non-responder' 'responder' 'responder' 'responder'
 'responder' 'responder' 'non-responder' 'responder' 'responder'
 'non-responder' 'responder' 'responder' 'non-responder' 'responder'
 'responder' 'non-responder' 'responder' 'responder' 'non-responder'
 'responder' 'responder' 'responder' 'responder' 'non-responder'
 'responder' 'responder' 'non-responder' 'responder' 'responder'
 'non-responder' 'responder' 'responder' 'responder' 'responder'
 'responder' 'non-responder' 'non-responder' 'non-responder' 'responder'
 'responder' 'responder' 'non-responder' 'non-responder' 'responder'
 'responder' 'responder' 'responder' 'non-responder' 'non-responder'
 'respond

**KNeighbors - Método dos k vizinhos mais próximos**

In [12]:
from sklearn.neighbors import KNeighborsClassifier

def knn_model_training(train_in, train_out, test_in, test_out):
    knn_model = KNeighborsClassifier()
    knn_model = knn_model.fit(train_in, train_out)
    knn_pred = knn_model.predict(test_in)
    print("\nKNeighbors")
    print("Valores previstos:\n" , knn_pred)
    print("Valores reais:\n" , test_out)
    return knn_model, knn_pred

knn_model, knn_pred = knn_model_training(train_in, train_out, test_in, test_out)


KNeighbors
Valores previstos:
 ['non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non-responder' 'non-responder' 'non-responder' 'non-responder'
 'non

#### **Avaliação do modelo**

In [13]:
def model_evaluation(test_out, tree_pred, svm_pred, gnb_pred, knn_pred):
    print("\nPercentagem de exemplos corretamente previstos")
    print("Decision Tree:", accuracy_score(test_out, tree_pred))
    print("SVMs:", accuracy_score(test_out, svm_pred))
    print("Naive Bayes:", accuracy_score(test_out, gnb_pred))
    print("Regressão Logística:", accuracy_score(test_out, logistic_pred))
    print("KNeighbors:", accuracy_score(test_out, knn_pred))

model_evaluation(test_out, tree_pred, svm_pred, gnb_pred, knn_pred)


Percentagem de exemplos corretamente previstos
Decision Tree: 0.6785714285714286
SVMs: 0.7619047619047619
Naive Bayes: 0.7619047619047619
Regressão Logística: 0.40476190476190477
KNeighbors: 0.7380952380952381


A partir dos resultados obtidos, podemos concluir que o modelos SVM e o modelo Naive Bayes são os que apresentam o melhor desempenho para prever a resposta de um indivíduo ao lithium, com uma percentagem de 76.19%.

O modelo KNeighbors também apresentou uma performance aceitável, com uma acurácia de cerca de 73.81%.

Por outro lado, o modelo de Regressão Logística apresentou o pior desempenho, com uma percentagem de apenas 40,47%. Isto pode indicar que este modelo não é adequado para esse tipo de problema ou que os dados não estão devidamente representados pela classe utilizada.

De seguida, vamos usar uma validação cruzada com, 5 partições, para analisar o desempenho dos dois melhores modelos a fim de ajudar a avaliar a estabilidade e a confiabilidade dos modelos, fornecendo uma estimativa mais precisa da performance deles em dados não vistos.

In [14]:
from sklearn.model_selection import cross_val_score

def model_cross_val_score(classe, modelA, modelB):
    meta_values = meta[classe].values

    print(modelA)
    scores = cross_val_score(modelA, input_sc, meta_values, cv = 5)
    print(scores)
    print(scores.mean())

    print(modelB)
    scores = cross_val_score(modelB, input_sc, meta_values, cv = 5)
    print(scores)
    print(scores.mean())

model_cross_val_score(classe, gnb_model, svm_model)

GaussianNB()
[0.83333333 0.83333333 0.75       0.75       0.79166667]
0.7916666666666667
SVC(C=100.0, gamma=0.001)
[0.79166667 0.79166667 0.79166667 0.79166667 0.75      ]
0.7833333333333333


A partir dos resultados da validação cruzada, podemos concluir que o melhor modelo para prever a resposta de um indivíduo ao lithium é o modelo Naive Baye, com uma acurácia média de aproximadamente 79.17%. Já o modelo SVC tem uma acurácia média de aproximadamente 78,33%.

Avaliando o desempenho dos dois melhores modelos na validação cruzada, vemos que ambos apresentam uma boa acurácia, o que sugere que os modelos foram bem treinados e que eles são capazes de prever a resposta de um indivíduo ao lithium com uma boa precisão. 

No entanto, apesar de terem acurácias elevadas, estas métricas podem não representar totalmente a qualidade do modelo, dado que, os valores apresentados são relativos à média de apenas cinco validações cruzadas, querendo isto dizer que a acurácia real do modelo pode ser diferente com outros conjuntos de dados.

**De seguida, vamos repetir as várias etapas para aprendizagem máquina para a analise das restantes classes.**

### Modelo de previsão baseada no tipo de agente (lithium ou controle) - classe "agent"

In [15]:
classe = "agent"
# separação dos dados para o treino e para o teste
train_in, train_out, test_in, test_out = splitData(classe)

# treino do modelo baseado nos seguintes algoritmos
# - Árvore de decisão
tree_model, tree_pred = tree_model_training(train_in, train_out, test_in, test_out)
# - SVM
svm_model, svm_pred = svm_model_training(train_in, train_out, test_in, test_out)
# - Naive Bayes
gnb_model, gnb_pred = gnb_model_training(train_in, train_out, test_in, test_out)
# - Regressão Logística
logistic_model, logistic_pred = logistic_model_training(train_in, train_out, test_in, test_out)
# - KNeighbors
knn_model, knn_pred = knn_model_training(train_in, train_out, test_in, test_out)

# avaliação dos modelos
model_evaluation(test_out, tree_pred, svm_pred, gnb_pred, knn_pred)

Separação dos dados para treino e teste
Input shape: (120, 22164)
Train in shape: (36, 22164)
Train out shape: (36,)
Test in shape: (84, 22164)
Test out shape: (84,)

Árvore de decisão
Valores previstos:  ['control' 'lithium' 'control' 'control' 'lithium' 'control' 'control'
 'lithium' 'control' 'control' 'control' 'lithium' 'lithium' 'lithium'
 'lithium' 'lithium' 'lithium' 'lithium' 'lithium' 'lithium' 'lithium'
 'control' 'lithium' 'control' 'control' 'lithium' 'lithium' 'lithium'
 'lithium' 'lithium' 'lithium' 'lithium' 'control' 'lithium' 'lithium'
 'lithium' 'control' 'lithium' 'control' 'control' 'lithium' 'control'
 'lithium' 'control' 'lithium' 'control' 'lithium' 'control' 'lithium'
 'lithium' 'control' 'control' 'lithium' 'lithium' 'lithium' 'lithium'
 'control' 'lithium' 'lithium' 'lithium' 'lithium' 'control' 'lithium'
 'lithium' 'lithium' 'control' 'lithium' 'lithium' 'lithium' 'lithium'
 'control' 'lithium' 'control' 'lithium' 'lithium' 'lithium' 'lithium'
 'lithium' 'co

A partir dos resultados, pode-se concluir que o modelo de Regressão Logística teve o melhor desempenho para prever o tipo de agente (lithium ou controle) com uma taxa de precisão de 55,71%. Logo de seguida o modelo de Árvore de Decisão com 55,71%. 

No entanto, estes resultados indicam que os modelos não foram capazes de prever corretamente a resposta de um indivíduo ao lithium. 

De seguida, vamos usar uma validação cruzada com, 5 partições, para analisar o desempenho dos dois melhores modelos.

In [16]:
model_cross_val_score(classe, logistic_model, tree_model)

LogisticRegression(C=100000.0, solver='liblinear')
[0.75       0.75       0.83333333 0.70833333 0.54166667]
0.7166666666666667
DecisionTreeClassifier()
[0.625      0.41666667 0.70833333 0.54166667 0.54166667]
0.5666666666666667


A partir dos resultados da validação cruzada, podemos concluir que o modelo de Regressão Logística tem um desempenho geral melhor do que o modelo de Árvore de Decisão, com uma média de acurácia de 71.67% comparada com 56.67%.
No entanto, as acurácias das previsões para cada partição da validação cruzada variam bastante, com valores que vão desde 41,67% até 70,83%. Isso indica que ambos os modelos precisam ser melhorados para tornarem-se mais precisos e confiáveis. Provavelmente seria necessário obter mais dados, com maior variação.

### Modelo de previsão baseada no momento da coleta da amostra - classe "time"

In [17]:
classe = "time"
# separação dos dados para o treino e para o teste
train_in, train_out, test_in, test_out = splitData(classe)

# treino do modelo baseado nos seguintes algoritmos
# - Árvore de decisão
tree_model, tree_pred = tree_model_training(train_in, train_out, test_in, test_out)
# - SVM
svm_model, svm_pred = svm_model_training(train_in, train_out, test_in, test_out)
# - Naive Bayes
gnb_model, gnb_pred = gnb_model_training(train_in, train_out, test_in, test_out)
# - Regressão Logística
logistic_model, logistic_pred = logistic_model_training(train_in, train_out, test_in, test_out)
# - KNeighbors
knn_model, knn_pred = knn_model_training(train_in, train_out, test_in, test_out)

# avaliação dos modelos
model_evaluation(test_out, tree_pred, svm_pred, gnb_pred, knn_pred)

Separação dos dados para treino e teste
Input shape: (120, 22164)
Train in shape: (36, 22164)
Train out shape: (36,)
Test in shape: (84, 22164)
Test out shape: (84,)

Árvore de decisão
Valores previstos:  ['baseline' 'baseline' 'baseline' 'baseline' '1 month' 'baseline'
 'baseline' '1 month' 'baseline' 'baseline' 'baseline' 'baseline'
 '1 month' '1 month' 'baseline' '1 month' 'baseline' 'baseline' '1 month'
 'baseline' '1 month' 'baseline' '1 month' 'baseline' '1 month' 'baseline'
 'baseline' 'baseline' 'baseline' '1 month' '1 month' 'baseline' '1 month'
 'baseline' 'baseline' 'baseline' 'baseline' 'baseline' 'baseline'
 'baseline' '1 month' 'baseline' 'baseline' '1 month' 'baseline' '1 month'
 'baseline' '1 month' 'baseline' 'baseline' 'baseline' 'baseline'
 'baseline' 'baseline' '1 month' 'baseline' 'baseline' '1 month'
 'baseline' 'baseline' 'baseline' 'baseline' 'baseline' 'baseline'
 '1 month' 'baseline' 'baseline' '1 month' 'baseline' 'baseline'
 'baseline' '1 month' 'baseline' '

Com base nos resultados dos modelos de previsão, podemos concluir que nenhum dos modelos teve desempenho consistentemente elevado na previsão do momento da coleta da amostra. Embora o SVM e Regressão Logística tenham tido o melhor desempenho satisfatório com 52.38% de exemplos corretamente previstos.

De seguida vamos usar uma validação cruzada com, 5 partições, para analisar o desempenho de dois dos melhores modelos.

In [20]:
model_cross_val_score(classe, svm_model, logistic_model)

SVC(C=100.0, gamma=0.001)
[0.5        0.45833333 0.58333333 0.5        0.625     ]
0.5333333333333333
LogisticRegression(C=100000.0, solver='liblinear')
[0.625      0.45833333 0.58333333 0.70833333 0.625     ]
0.6


A partir dos resultados da validação cruzada, podemos concluir que o melhor modelo para prever a resposta de um indivíduo a um determinado tratamento é o modelo de Regressão Logística, com uma acurácia média de aproximadamente 60%. Já o modelo SVC tem uma acurácia média de aproximadamente 53,33%. 

## Conclusão
A partir dos resultados apresentados, podemos concluir que a classe "other" teve o melhor desempenho em relação às outras classes, tendo o melhor modelo apresentado uma acurácia média de aproximadamente 79%.

De uma forma geral, podemos concluir que os modelos apresentaram desempenho variavel para as diferentes classes. Alguns modelos, como a regressão logística e a árvore de decisão, tiveram desempenho aceitável para a classe "agent", enquanto outros modelos, como o Naive Bayes e a SVM, apresentaram melhor desempenho para a classe "other". Para a classe "time", o desempenho dos modelos foi moderado. 

No entanto, é importante referir que estes resultados são baseados numa única amostra de dados com poucos dados e que podem não representar a verdadeira performance dos modelos com dados futuros.